In [ ]:
import cooltools
from collections import defaultdict
import cooler
import numpy as np
import pandas as pd

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [ ]:
regions = pd.read_csv('/mnt/md0/clarice/src/region_idx.txt', sep='\t')

Example with GM12878 RCMC vs holdout predictions, can be used for all other coolers

In [ ]:
res = 500

gm12878_rcmc = cooler.Cooler('/mnt/md0/clarice/fan_prediction_analyses/RCMC_matrices_to_clrs/GM12878/GM12878_500kb_RCMC_smoothed.mcool::resolutions/' + str(res))
gm12878_pred = cooler.Cooler('/mnt/md0/clarice/fan_prediction_analyses/holdout_model_coolers/25_GM_newTestSet_43/25_GM_newTestSet_43_merged.mcool::resolutions/' + str(res))

In [ ]:
def calculate_corrs(rcmc_clr, pred_clr, region, res=500, max_range=1000):

    distance_corr = defaultdict(list)

    rcmc_mat = rcmc_clr.matrix(balance=False).fetch(region)
    pred_mat = pred_clr.matrix(balance=False).fetch(region)

    for offset in range(20, max_range):
        distance = offset*res
        rcmc_diag = cooltools.lib.numutils.get_diag(rcmc_mat, i=offset)
        pred_diag = cooltools.lib.numutils.get_diag(pred_mat, i=offset)
        current_df = pd.DataFrame({'rcmc': rcmc_diag, 'pred': pred_diag})
        current_df = current_df[current_df['pred'] != 0]
        correlation = current_df['rcmc'].corr(current_df['pred'], method = 'pearson')
        distance_corr[distance].append(correlation)
    df = pd.DataFrame.from_dict(distance_corr, orient='index', columns=['corr'])
    df['distance'] = df.index

    return df
